In [1]:
#!/usr/bin/env bash

<span style="color: red; font-weight: bold">Use the following command to launch a server for this notebook:</span>

```bash
git clone git@github.com:input-output-hk/marlowe-cardano.git
cd marlowe-cardano/marlowe-runtime/
nix run ../marlowe-cli
```

Then navigate to the `examples/` folder in Jupyter and open this notebook.

# Demonstration of Marlowe Runtime Capabilities

**Executive Summary**

This contract for a token sale uses nearly all of the features of the Marlowe Runtime backend. The `marlowe` commands `create`, `deposit`, `choose`, `notify`, and `advance` transition the contract, and the command `withdraw` redeems funds paid by the contract. It can be run using addresses for the parties, role tokens for the parties, or a mixture of addresses and role tokens for the parties.


**Parties**

* The *Offeror* is selling `Marlon` tokens.
* The *Purchaser* bids Ada for purchasing the `Marlon` tokens.
* The *Approver* has the option to block the transaction, perhaps because KYC requirements have not been met by the *Offeror* or *Purchaser*.


**Scenario**

1. The *Approver* creates the contract.
2. The *Offeror* chooses to offer 500 `Marlon` tokens.
3. The *Offeror* deposits the 500 `Marlon` tokens that the chose to offer.
4. The *Purchaser* bids 100 Ada for the `Marlon` tokens.
5. The *Offeror* accepts the bid made by the *Purchaser*.
6. The *Purchaser* deposits the 100 Ada that they bid.
7. The *Approver* declines to forbid the transaction by waiting for the next timeout in the contract, triggering the payment of the `Marlon` tokens to the *Purchaser*.
8. The contract is notified to close, paying the Ada to the *Offeror*.


![Token-Bid Contract](token-bid.png)

## Preliminaries

Record version numbers

In [2]:
marlowe-cli --version

marlowe-cli 0.0.10.0


In [3]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [4]:
git rev-parse HEAD

cdf9a03d2d61db0e057951503e2c26e4be434735


### Setup the faucet.

Set the location of keys.

In [5]:
TREASURY=/extra/iohk/networks/treasury

Set the faucet.

In [6]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select network

In a separate terminal, set up a tunnel to the Marlowe Runtime development server:
```bash
rm /tmp/preview.socket
ssh -NT \
  -L/tmp/preview.socket:/data/networks/preview/node.socket \
  -L 3717:127.0.0.1:23717 \
  -L 3718:127.0.0.1:23718 \
  -L 3719:127.0.0.1:23719 \
  -L 3721:127.0.0.1:23721 \
  -L 3723:127.0.0.1:23723 \
  54.202.238.5                                              
```

In [7]:
if false
then

export CARDANO_NODE_SOCKET_PATH=/tmp/preview.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)

else

source /extra/iohk/networks/preview/configure.env

fi

echo "${MAGIC[@]}"

--testnet-magic 2


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [8]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Offeror

In [9]:
OFFEROR_SKEY="$TREASURY/john-fletcher.skey"
OFFEROR_VKEY="$TREASURY/john-fletcher.vkey"

Create the token offeror's keys, if necessary.

In [10]:
if [[ ! -e "$OFFEROR_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$OFFEROR_SKEY" --verification-key-file "$OFFEROR_VKEY"
fi
OFFEROR_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$OFFEROR_VKEY")
echo "$OFFEROR_ADDR"

addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r


Fund the address.

In [11]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600 \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$OFFEROR_ADDR"

TxId "03b694b7cafaf03f0ca4354092f9528aa8bb30c13ae8bfad236f311406d6ec59"


#### The Purchaser

In [12]:
PURCHASER_SKEY="$TREASURY/thomas-kyd.skey"
PURCHASER_VKEY="$TREASURY/thomas-kyd.vkey"

Create the token purchaser's keys, if necessary.

In [13]:
if [[ ! -e "$PURCHASER_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PURCHASER_SKEY" --verification-key-file "$PURCHASER_VKEY"
fi
PURCHASER_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PURCHASER_VKEY")
echo "$PURCHASER_ADDR"

addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5


Fund the address.

In [14]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600  \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$PURCHASER_ADDR"

TxId "894fce7cdc7917249b458e42236e62da9751824c1618b4bac6a33047e4a93d46"


#### The Approver

In [15]:
APPROVER_SKEY="$TREASURY/john-webster.skey"
APPROVER_VKEY="$TREASURY/john-webster.vkey"

Create the approvers's keys, if necessary.

In [16]:
if [[ ! -e "$APPROVER_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$APPROVER_SKEY" --verification-key-file "$APPROVER_VKEY"
fi
APPROVER_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$APPROVER_VKEY")
echo "$APPROVER_ADDR"

addr_test1vp6g96pgt37le7lc0wew60shsdh2lmqrt57da9t6k64sltgasvxlp


Fund the address.

In [17]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600  \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$APPROVER_ADDR"

TxId "59df65b6353f907eaad843bc70746422f3e71eef1a2ef541fa973b5f11aeb6c7"


### Mint the tokens to be purchased

In [18]:
TOKEN_NAME=Marlon

In [19]:
POLICY_ID=$(
marlowe-cli util mint \
  --issuer "$FAUCET_ADDR:$FAUCET_SKEY" \
  --count 1000 \
  --out-file /dev/null \
  --submit 600 \
  "$TOKEN_NAME:$OFFEROR_ADDR" \
| sed -e 's/^PolicyID "\(.*\)"$/\1/' \
)
echo "POLICY_ID = $POLICY_ID"


Fee: Lovelace 172893
Size: 256 / 16384 = 1%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
POLICY_ID = 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d


In [20]:
TOKEN="$POLICY_ID.$TOKEN_NAME"
echo "TOKEN = $TOKEN"

TOKEN = 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.Marlon


### Time computations

In [22]:
SECOND=1000
MINUTE=$((60 * SECOND))

In [23]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1666975290000


## The Contract

We set the parameters for the contract.

In [24]:
ADA=1000000

MINIMUM_ADA=$((3 * ADA))
MAXIMUM_TOKEN=1000
MAXIMUM_ADA=$((200 * ADA))

TOKEN_DEADLINE=$((NOW + 14 * MINUTE))
BID_DEADLINE=$((NOW + 15 * MINUTE))
ACCEPT_DEADLINE=$((NOW + 16 * MINUTE))
ADA_DEADLINE=$((NOW + 17 * MINUTE))
FORBID_DEADLINE=$((NOW + 18 * MINUTE))
NOTIFY_DEADLINE=$((NOW + 30 * MINUTE))

# Select either a pure address-based contract, or a mixed role and address one.
case "addresses" in
  # Addresses only
  addresses)
    OFFEROR_PARTY="$OFFEROR_ADDR"
    PURCHASER_PARTY="$PURCHASER_ADDR"
    APPROVER_PARTY="$APPROVER_ADDR"
    OFFEROR_PARTY_C="address: $OFFEROR_ADDR"
    PURCHASER_PARTY_C="address: $PURCHASER_ADDR"
    APPROVER_PARTY_C="address: $APPROVER_ADDR"
    ;;
  # Roles only
  roles)
    OFFEROR_PARTY="Offeror"
    PURCHASER_PARTY="Purchaser"
    APPROVER_PARTY="Approver"
    OFFEROR_PARTY_C="role_token: Offeror"
    PURCHASER_PARTY_C="role_token: Purchaser"
    APPROVER_PARTY_C="role_token: Approver"
    ;;
  # Mixture of addresses and roles
  *)
    OFFEROR_PARTY="$OFFEROR_ADDR"
    PURCHASER_PARTY="Purchaser"
    APPROVER_PARTY="Approver"
    OFFEROR_PARTY_C="address: $OFFEROR_ADDR"
    PURCHASER_PARTY_C="role_token: Purchaser"
    APPROVER_PARTY_C="role_token: Approver"
    ;;
esac

Insert the parameters into the contract.

In [25]:
yaml2json << EOI > token-bid-1.contract
when:
- case:
    choose_between:
    - from: 1
      to: $MAXIMUM_TOKEN
    for_choice:
      choice_name: Offer
      choice_owner:
        $OFFEROR_PARTY_C
  then:
    when:
    - case:
        deposits:
          value_of_choice:
            choice_name: Offer
            choice_owner:
              $OFFEROR_PARTY_C
        into_account:
          $OFFEROR_PARTY_C
        of_token:
          currency_symbol: $POLICY_ID
          token_name: $TOKEN_NAME
        party:
          $OFFEROR_PARTY_C
      then:
        when:
        - case:
            choose_between:
            - from: 1
              to: $MAXIMUM_ADA
            for_choice:
              choice_name: Bid
              choice_owner:
                $PURCHASER_PARTY_C
          then:
            when:
            - case:
                choose_between:
                - from: 1
                  to: 1
                for_choice:
                  choice_name: Accept
                  choice_owner:
                    $OFFEROR_PARTY_C
              then:
                when:
                - case:
                    deposits:
                      value_of_choice:
                        choice_name: Bid
                        choice_owner:
                          $PURCHASER_PARTY_C
                    into_account:
                      $PURCHASER_PARTY_C
                    of_token:
                      currency_symbol: ''
                      token_name: ''
                    party:
                      $PURCHASER_PARTY_C
                  then:
                    when:
                    - case:
                        choose_between:
                        - from: 1
                          to: 1
                        for_choice:
                          choice_name: Forbid
                          choice_owner:
                            $APPROVER_PARTY_C
                      then: close
                    timeout: $FORBID_DEADLINE
                    timeout_continuation:
                      pay:
                        amount_of_token:
                          currency_symbol: ''
                          token_name: ''
                        in_account:
                          $PURCHASER_PARTY_C
                      from_account:
                        $PURCHASER_PARTY_C
                      token:
                        currency_symbol: ''
                        token_name: ''
                      to:
                        account:
                          $OFFEROR_PARTY_C
                      then:
                        pay:
                          amount_of_token:
                            currency_symbol: $POLICY_ID
                            token_name: $TOKEN_NAME
                          in_account:
                            $OFFEROR_PARTY_C
                        from_account:
                          $OFFEROR_PARTY_C
                        token:
                          currency_symbol: $POLICY_ID
                          token_name: $TOKEN_NAME
                        to:
                          party:
                            $PURCHASER_PARTY_C
                        then:
                          when:
                          - case:
                              notify_if: true
                            then: close
                          timeout: $NOTIFY_DEADLINE
                          timeout_continuation: close
                timeout: $ADA_DEADLINE
                timeout_continuation: close
            timeout: $ACCEPT_DEADLINE
            timeout_continuation: close
        timeout: $BID_DEADLINE
        timeout_continuation: close
    timeout: $TOKEN_DEADLINE
    timeout_continuation: close
timeout: $TOKEN_DEADLINE
timeout_continuation: close
EOI
cat token-bid-1.contract

{"timeout":1666976130000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":1,"to":1000}],"for_choice":{"choice_name":"Offer","choice_owner":{"address":"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"}}},"then":{"timeout":1666976130000,"timeout_continuation":"close","when":[{"case":{"deposits":{"value_of_choice":{"choice_name":"Offer","choice_owner":{"address":"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"}}},"into_account":{"address":"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"},"of_token":{"currency_symbol":"8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d","token_name":"Marlon"},"party":{"address":"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"}},"then":{"timeout":1666976190000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":1,"to":200000000}],"for_choice":{"choice_name":"Bid","choice_owner":{"address":"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m

## Simulate the Contract

Simulate the contract before running it on chain.

We start the contract with an initial state of the minimum Ada.

In [26]:
yaml2json << EOI > token-bid-1.state
accounts:
- - - address: $APPROVER_ADDR
    - currency_symbol: ''
      token_name: ''
  - $MINIMUM_ADA
boundValues: []
choices: []
minTime: 1
EOI
cat token-bid-1.state

{"accounts":[[[{"address":"addr_test1vp6g96pgt37le7lc0wew60shsdh2lmqrt57da9t6k64sltgasvxlp"},{"currency_symbol":"","token_name":""}],3000000]],"boundValues":[],"choices":[],"minTime":1}


### Transaction 1. Create the contract

In [56]:
marlowe-cli run initialize \
  --contract-file token-bid-1.contract \
  --state-file    token-bid-1.state \
  --out-file      token-bid-1.marlowe \
  --roles-currency "00000000000000000000000000000000000000000000000000000000" \
  --permanently-without-staking \
  --print-stats


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"

Validator size: 12505
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}


### Transaction 2. The Offeror offers tokens

In [57]:
marlowe-cli run prepare \
  --marlowe-file token-bid-1.marlowe \
  --out-file     token-bid-2.marlowe \
  --choice-party "$OFFEROR_PARTY" \
  --choice-name Offer \
  --choice-number "$((MAXIMUM_TOKEN / 2))" \
  --invalid-before    "$((NOW + 1 * MINUTE))" \
  --invalid-hereafter "$((NOW + 2 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666975350000},POSIXTime {getPOSIXTime = 1666975410999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666975350000},POSIXTime {getPOSIXTime = 1666975410999}), txInputs = [NormalInput (IChoice (ChoiceId "Offer" "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"") 500)]}

Datum size: 1314


### Transaction 3. The Offeror deposits the offered tokens

In [58]:
marlowe-cli run prepare \
  --marlowe-file token-bid-2.marlowe \
  --out-file     token-bid-3.marlowe \
  --deposit-account "$OFFEROR_PARTY" \
  --deposit-party "$OFFEROR_PARTY" \
  --deposit-amount "$((MAXIMUM_TOKEN / 2))" \
  --deposit-token "$TOKEN" \
  --invalid-before    "$((NOW + 1 * MINUTE))" \
  --invalid-hereafter "$((NOW + 2 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666975350000},POSIXTime {getPOSIXTime = 1666975410999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666975350000},POSIXTime {getPOSIXTime = 1666975410999}), txInputs = [NormalInput (IDeposit "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"" "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"" (Token "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d" "Marlon") 500)]}

Datum size: 1184


### Transaction 4. The Purchaser makes a bid

In [59]:
marlowe-cli run prepare \
  --marlowe-file token-bid-3.marlowe \
  --out-file     token-bid-4.marlowe \
  --choice-party "$PURCHASER_PARTY" \
  --choice-name Bid \
  --choice-number "$((MAXIMUM_ADA / 2))" \
  --invalid-before    "$((NOW + 1 * MINUTE))" \
  --invalid-hereafter "$((NOW + 2 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666975350000},POSIXTime {getPOSIXTime = 1666975410999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666975350000},POSIXTime {getPOSIXTime = 1666975410999}), txInputs = [NormalInput (IChoice (ChoiceId "Bid" "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"") 100000000)]}

Datum size: 1151


### Transaction 5. The Offeror accepts the Purchaser's bid

In [60]:
marlowe-cli run prepare \
  --marlowe-file token-bid-4.marlowe \
  --out-file     token-bid-5.marlowe \
  --choice-party "$OFFEROR_PARTY" \
  --choice-name Accept \
  --choice-number 1 \
  --invalid-before    "$((NOW + 1 * MINUTE))" \
  --invalid-hereafter "$((NOW + 2 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666975350000},POSIXTime {getPOSIXTime = 1666975410999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666975350000},POSIXTime {getPOSIXTime = 1666975410999}), txInputs = [NormalInput (IChoice (ChoiceId "Accept" "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"") 1)]}

Datum size: 1118


### Transaction 6. The Purchaser deposits the Ada that they had bid

In [61]:
marlowe-cli run prepare \
  --marlowe-file token-bid-5.marlowe \
  --out-file     token-bid-6.marlowe \
  --deposit-account "$PURCHASER_PARTY" \
  --deposit-party "$PURCHASER_PARTY" \
  --deposit-amount "$((MAXIMUM_ADA / 2))" \
  --invalid-before    "$((NOW + 1 * MINUTE))" \
  --invalid-hereafter "$((NOW + 2 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666975350000},POSIXTime {getPOSIXTime = 1666975410999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666975350000},POSIXTime {getPOSIXTime = 1666975410999}), txInputs = [NormalInput (IDeposit "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" (Token "" "") 100000000)]}

Datum size: 992


### Transaction 7. The Approver declines to forbid the sale by lettting the timeout pass

In [62]:
marlowe-cli run prepare \
  --marlowe-file token-bid-6.marlowe \
  --out-file     token-bid-7.marlowe \
  --invalid-before    "$((NOW + 20 * MINUTE))" \
  --invalid-hereafter "$((NOW + 21 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666976490000},POSIXTime {getPOSIXTime = 1666976550999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666976490000},POSIXTime {getPOSIXTime = 1666976550999}), txInputs = []}

Datum size: 397
Payment 1
  Acccount: "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Payee: Account "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Ada: Lovelace {getLovelace = 100000000}
Payment 2
  Acccount: "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Payee: Party "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Ada: Lovelace {getLovelace = 0}
  8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d."Marlon": 500


### Transaction 8. The contract is notified to make the final payment

In [63]:
marlowe-cli run prepare \
  --marlowe-file token-bid-7.marlowe \
  --out-file     token-bid-8.marlowe \
  --notify \
  --invalid-before    "$((NOW + 22 * MINUTE))" \
  --invalid-hereafter "$((NOW + 23 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666976610000},POSIXTime {getPOSIXTime = 1666976670999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666976610000},POSIXTime {getPOSIXTime = 1666976670999}), txInputs = [NormalInput INotify]}

Datum size: 241
Payment 1
  Acccount: "\"addr_test1vp6g96pgt37le7lc0wew60shsdh2lmqrt57da9t6k64sltgasvxlp\""
  Payee: Party "\"addr_test1vp6g96pgt37le7lc0wew60shsdh2lmqrt57da9t6k64sltgasvxlp\""
  Ada: Lovelace {getLovelace = 3000000}
Payment 2
  Acccount: "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Payee: Party "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Ada: Lovelace {getLovelace = 100000000}


## Run the Contract

### Initial Funds

In [65]:
cardano-cli query utxo "${MAGIC[@]}" --address "$OFFEROR_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


In [66]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PURCHASER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


In [69]:
cardano-cli query utxo "${MAGIC[@]}" --address "$APPROVER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


### Transaction 1. Create the contract

Build the transaction.

In [28]:
TX_1=$(
marlowe create \
  --core-file token-bid-1.contract \
  --min-utxo "$MINIMUM_ADA" \
  --change-address "$APPROVER_ADDR" \
  --manual-sign token-bid-1.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
CONTRACT_ID="$TX_1"
echo "CONTRACT_ID = TX_1 = $CONTRACT_ID"

CONTRACT_ID = TX_1 = 473b56fc50bfe1bf25d1c34000c462f40ed5e93daa2bdddd583309f61d9172db#1


Sign the transaction.

In [29]:
cardano-cli transaction sign \
  --tx-body-file token-bid-1.txbody \
  --out-file     token-bid-1.tx \
  --signing-key-file "$APPROVER_SKEY"

Submit the transaction using Marlowe Runtime.

In [30]:
marlowe submit token-bid-1.tx

"{\"blockHeaderHash\":\"91afad27d15cbd9fa735214b7e41dca89eb90d55227f3f03bb545c32188e0b18\",\"blockNo\":322346,\"slotNo\":6972195}"


### Transaction 2. The Offeror offers tokens

Build the transaction.

In [31]:
TX_2=$(
marlowe choose \
  --contract "$CONTRACT_ID" \
  --party "$OFFEROR_PARTY" \
  --choice Offer \
  --value "$((MAXIMUM_TOKEN / 2))" \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 1 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --change-address "$OFFEROR_ADDR" \
  --manual-sign token-bid-2.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_2 = $TX_2"

TX_2 = 29bdc2d3531c343f0a96c571f6a380fa8ab4a3dc69d80a2c114c355b13d0006f


Sign the transaction.

In [32]:
cardano-cli transaction sign \
  --tx-body-file token-bid-2.txbody \
  --out-file     token-bid-2.tx \
  --signing-key-file "$OFFEROR_SKEY"

Submit the transaction using Marlowe Runtime.

In [33]:
marlowe submit token-bid-2.tx

"{\"blockHeaderHash\":\"99649384ac5dcda597201d69182afa19019ca11081810c3a1d7d0e94d167f905\",\"blockNo\":322347,\"slotNo\":6972205}"


### Transaction 3. The Offeror deposits the offered tokens

Build the transaction.

In [34]:
TX_3=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --to-party "$OFFEROR_PARTY" \
  --from-party "$OFFEROR_PARTY" \
  --quantity "$((MAXIMUM_TOKEN / 2))" \
  --currency "$POLICY_ID" \
  --token-name "$TOKEN_NAME" \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 1 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --change-address "$OFFEROR_ADDR" \
  --manual-sign token-bid-3.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_3 = $TX_3"

TX_3 = d173519ccbcf86b6c92a285122a0888393539099fa49a2472fe8e2992dcb2f77


Sign the transaction.

In [35]:
cardano-cli transaction sign \
  --tx-body-file token-bid-3.txbody \
  --out-file     token-bid-3.tx \
  --signing-key-file "$OFFEROR_SKEY"

Submit the transaction using Marlowe Runtime.

In [36]:
marlowe submit token-bid-3.tx

"{\"blockHeaderHash\":\"ba1652fc7fddd716dbe8a4170b1a394299efbec72d434f2a210c6abab0412b96\",\"blockNo\":322348,\"slotNo\":6972256}"


### Transaction 4. The Purchaser makes a bid

Build the transaction.

In [37]:
TX_4=$(
marlowe choose \
  --contract "$CONTRACT_ID" \
  --party "$PURCHASER_PARTY" \
  --choice Bid \
  --value "$((MAXIMUM_ADA / 2))" \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 1 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --change-address "$PURCHASER_ADDR" \
  --manual-sign token-bid-4.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_4 = $TX_4"

TX_4 = 67cda92f8558ae18b54f7cecdae4b8a0c15a424cc691777739acf85c14c3475c


Sign the transaction.

In [38]:
cardano-cli transaction sign \
  --tx-body-file token-bid-4.txbody \
  --out-file     token-bid-4.tx \
  --signing-key-file "$PURCHASER_SKEY"

Submit the transaction using Marlowe Runtime.

In [39]:
marlowe submit token-bid-4.tx

"{\"blockHeaderHash\":\"569595d8ad20bf0dc9eb4568dea77387feb6b1cb74ae046f425b0c886bc2ece4\",\"blockNo\":322350,\"slotNo\":6972282}"


### Transaction 5. The Offeror accepts the Purchaser's bid

Build the transaction.

In [40]:
TX_5=$(
marlowe choose \
  --contract "$CONTRACT_ID" \
  --party "$OFFEROR_PARTY" \
  --choice Accept \
  --value 1 \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 1 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --change-address "$OFFEROR_ADDR" \
  --manual-sign token-bid-5.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_5 = $TX_5"

TX_5 = 782adbd5d1ffefdb778048aafaaed61dfd617ed2d92d528afc656ea285859261


Sign the transaction.

In [41]:
cardano-cli transaction sign \
  --tx-body-file token-bid-5.txbody \
  --out-file     token-bid-5.tx \
  --signing-key-file "$OFFEROR_SKEY"

Submit the transaction using Marlowe Runtime.

In [42]:
marlowe submit token-bid-5.tx

"{\"blockHeaderHash\":\"1d901893b1c2f569cae3dab4d7b1ca0707a87f258bb72ffab839de82d877d268\",\"blockNo\":322351,\"slotNo\":6972315}"


### Transaction 6. The Purchaser deposits the Ada that they had bid

Build the transaction.

In [43]:
TX_6=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --to-party "$PURCHASER_PARTY" \
  --from-party "$PURCHASER_PARTY" \
  --lovelace "$((MAXIMUM_ADA / 2))" \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 1 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --change-address "$PURCHASER_ADDR" \
  --manual-sign token-bid-7.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_6 = $TX_6"

TX_6 = e3f4f1ba3ec2bb9213394f16e2fca99bbce8b9efa2e84f38fdcc6cf0edcb2b06


Sign the transaction.

In [44]:
cardano-cli transaction sign \
  --tx-body-file token-bid-7.txbody \
  --out-file     token-bid-7.tx \
  --signing-key-file "$PURCHASER_SKEY"

Submit the transaction using Marlowe Runtime.

In [45]:
marlowe submit token-bid-7.tx

"{\"blockHeaderHash\":\"6c79b1158487e20a4b4467182b50e22873791a9847f9eff623c47129b6b79824\",\"blockNo\":322352,\"slotNo\":6972324}"


### Transaction 7. The Approver declines to forbid the sale by lettting the timeout pass

Wait fo the timeout, if necessary.

In [46]:
WAIT="$(((FORBID_DEADLINE - $(date -u +%s) * SECOND + 1 * MINUTE + 1 * SECOND) / SECOND))"
if [[ $WAIT -ge 0 ]]
then
  echo "Waiting $WAIT seconds before the timeout passes."
  sleep "$WAIT"
fi

Waiting 900 seconds before the timeout passes.


Build the transaction.

In [47]:
TX_7=$(
marlowe advance \
  --contract "$CONTRACT_ID" \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 1 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --change-address "$APPROVER_ADDR" \
  --manual-sign token-bid-7.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_7 = $TX_7"

TX_7 = da89a5f0b5d74fee566d20d15f7a37682b543729d30a96ab716645c7a6d05759


Sign the transaction.

In [48]:
cardano-cli transaction sign \
  --tx-body-file token-bid-7.txbody \
  --out-file     token-bid-7.tx \
  --signing-key-file "$APPROVER_SKEY"

Submit the transaction using Marlowe Runtime.

In [49]:
marlowe submit token-bid-7.tx

"{\"blockHeaderHash\":\"84e07ce123d1776b1b8828e1d2b1190ae631f3339a0e5cf779193c332440fd41\",\"blockNo\":322392,\"slotNo\":6973243}"


### Transaction 8. The contract is notified to make the final payment

Build the transaction.

In [50]:
TX_8=$(
marlowe notify \
  --contract "$CONTRACT_ID" \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 1 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --change-address "$OFFEROR_ADDR" \
  --manual-sign token-bid-8.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_8 = $TX_8"

TX_8 = 9e28adf63f8e4a5005ef3263e4e16e202463846144670f3ab1d47ef5293ad567


Sign the transaction.

In [51]:
cardano-cli transaction sign \
  --tx-body-file token-bid-8.txbody \
  --out-file     token-bid-8.tx \
  --signing-key-file "$OFFEROR_SKEY"

Submit the transaction using Marlowe Runtime.

In [52]:
marlowe submit token-bid-8.tx

"{\"blockHeaderHash\":\"7e2f06ee73af05edc928dd02d4457431d7049e0a0d46fa7a6368be220cc81c6a\",\"blockNo\":322393,\"slotNo\":6973302}"


### Final Funds

In [65]:
cardano-cli query utxo "${MAGIC[@]}" --address "$OFFEROR_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


In [66]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PURCHASER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


In [69]:
cardano-cli query utxo "${MAGIC[@]}" --address "$APPROVER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
